In [1]:
import os
import asyncio
import nest_asyncio
import requests

from dotenv import load_dotenv
from openai import OpenAI
from openai import AsyncOpenAI


# 加载环境变量
load_dotenv()
if not os.getenv("GITHUB_TOKEN"):
    print("Error: GITHUB_TOKEN is not set in .env file")
    exit(1)

GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")

# 测试模型列表
BASE_URL = "https://models.github.ai/inference"
headers = {
    "Authorization": f"Bearer {os.getenv('GITHUB_TOKEN')}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
}

# 初始化 OpenAI 客户端
MODEL_ID = "openai/gpt-4o-mini"


In [2]:
client = OpenAI(api_key=GITHUB_TOKEN, base_url=BASE_URL)

# 调用 LLM
try:
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": "用中文讲个冷笑话"}],
        temperature=0.2,
    )
    print("LLM response:", response.choices[0].message.content)
except Exception as e:
    print("LLM error:", e)

LLM response: 为什么鸡过马路？

因为它想去对面看看“鸡”是什么样的！


In [ ]:
# Apply nest_asyncio to allow async in Jupyter Notebook
nest_asyncio.apply()

client = AsyncOpenAI(api_key=GITHUB_TOKEN, base_url=BASE_URL)

# Define a list of prompts for batch processing
prompts = [
    "用中文讲一个冷笑话",
    "再讲一个冷笑话，要跟动物有关",
    "讲一个跟程序员有关的冷笑话",
    "讲一个简短的冷笑话",
]

# Asynchronous function to call LLM for a single prompt
async def call_llm(prompt):
    try:
        response = await client.chat.completions.create(
            model=MODEL_ID,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
        )
        return {"prompt": prompt, "response": response.choices[0].message.content}
    except Exception as e:
        return {"prompt": prompt, "response": None, "error": str(e)}


# Run batch calls asynchronously
async def main():
    tasks = [call_llm(prompt) for prompt in prompts]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    for result in results:
        print(f"Prompt: {result['prompt']}")
        if result.get("response"):
            print(f"Response: {result['response']}\n")
        else:
            print(f"Error: {result['error']}\n")
    return results

# Execute async batch calls in Jupyter
results = asyncio.run(main())
print("Batch results:", results)

In [ ]:
# Importing required libraries
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    api_key=GITHUB_TOKEN,
    model=MODEL_ID,
    base_url=BASE_URL,
    temperature=0.2,
)
prompt_template = ChatPromptTemplate.from_messages([
    ("user", "{input}")
])
chain = prompt_template | llm | StrOutputParser()

# Define prompts for batch processing
prompts = [
   "用中文讲一个幽默的笑话",
    "讲一个关于动物的幽默笑话",
    "讲一个与程序员相关的幽默笑话",
    "讲一个简短的幽默笑话",
]

# Asynchronous batch calling with LCEL
async def main():
    results = []
    try:
        responses = await chain.abatch([{"input": prompt} for prompt in prompts])
        results = [{"prompt": prompt, "response": response} for prompt, response in zip(prompts, responses)]
        for result in results:
            print(f"Prompt: {result['prompt']}")
            print(f"Response: {result['response']}\n")
    except Exception as e:
        print("Batch error:", str(e))
        results.append({"prompt": "batch", "response": None, "error": str(e)})
    return results

# Run async batch in Jupyter
results = asyncio.run(main())
print("Batch results:", results)